<a href="https://colab.research.google.com/github/DaniChinwendu/COMPUTATIONAL-CHEMISTRY-CHEMINFORMATICS/blob/main/PUBMED_AGENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')


Saving ALL DATA.csv to ALL DATA.csv
User uploaded file "ALL DATA.csv" with length 308990 bytes


In [ ]:
# Cell 1: Install (alternative with pinned versions)
# ============================================================================
import subprocess
subprocess.check_call(['pip', 'install', '-q',
    'langchain>=0.3.0',
    'langchain-core>=0.3.0',
    'langchain-google-genai>=2.0.0',
    'langgraph>=0.2.0',
    'pydantic>=2.0',
    'pandas',
    'google-generativeai',
    'requests'])

0

In [ ]:
# ============================================================================
# Cell 2: Setup - CHANGE YOUR EMAIL HERE!!!
# ============================================================================
import os, json, re, time, requests, pandas as pd
from typing import TypedDict, Annotated, Sequence, List, Dict, Any
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
os.environ["NCBI_EMAIL"] = "Ukaegbudaniel33@gmail.com"  # ← CHANGE THIS!

print(f"✅ Email: {os.getenv('NCBI_EMAIL')}")



In [ ]:
# ============================================================================
# Cell 3: State
# ============================================================================
class CompoundState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    compound_name: str
    search_query: str
    pmids: List[str]
    articles: List[Dict[str, Any]]
    interactions: List[Dict[str, Any]]


In [ ]:
# ============================================================================
# Cell 4: Tools
# ============================================================================
import xml.etree.ElementTree as ET

@tool
def search_pubmed(compound: str, max_results: int = 30) -> str:
    """Search PubMed for articles on a chemical compound with toxicology focus.

    Args:
        compound: The search query for PubMed (can include boolean operators)
        max_results: Maximum number of results to return (default: 30)

    Returns:
        JSON string with list of PubMed IDs (pmids)
    """
    try:
        email = os.getenv("NCBI_EMAIL")
        if not email or "example" in email:
            raise ValueError("❌ REPLACE YOUR EMAIL IN THE CODE!")
        url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        params = {"db": "pubmed", "term": compound, "retmax": min(max_results, 100),
                  "rettype": "xml", "tool": "agent", "email": email}
        response = requests.get(url, params=params, timeout=20)

        root = ET.fromstring(response.text)
        pmids = [id_elem.text for id_elem in root.findall(".//Id")]

        print(f"  ✓ Found {len(pmids)} articles")
        return json.dumps({"pmids": pmids})
    except Exception as e:
        print(f"  ❌ {e}")
        return json.dumps({"pmids": []})

@tool
def fetch_pubmed_articles(pmids_list: str) -> str:
    """Fetch full article metadata from PubMed using PMIDs.

    Args:
        pmids_list: JSON string containing list of PubMed IDs

    Returns:
        JSON string with article titles and abstracts
    """
    try:
        pmids = json.loads(pmids_list)
        if not pmids:
            return json.dumps({"articles": []})
        email = os.getenv("NCBI_EMAIL")
        articles = []
        for i in range(0, len(pmids), 50):
            batch = pmids[i:i+50]
            url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
            params = {"db": "pubmed", "id": ",".join(batch), "rettype": "xml", "email": email}
            response = requests.get(url, params=params, timeout=20)

            root = ET.fromstring(response.text)
            for article in root.findall(".//PubmedArticle"):
                pmid_elem = article.find(".//PMID")
                title_elem = article.find(".//ArticleTitle")
                abstract_elem = article.find(".//AbstractText")

                pmid = pmid_elem.text if pmid_elem is not None else ""
                title = title_elem.text if title_elem is not None else ""
                abstract = abstract_elem.text if abstract_elem is not None else ""

                if pmid and abstract:
                    articles.append({"pmid": pmid, "title": title, "abstract": abstract})

            time.sleep(0.5)
        print(f"  ✓ Fetched {len(articles)} articles")
        return json.dumps({"articles": articles})
    except Exception as e:
        print(f"  ❌ {e}")
        return json.dumps({"articles": []})

@tool
def extract_with_llm(article_data: str) -> str:
    """Extract molecular interactions and toxicological mechanisms from an article abstract using LLM.

    Args:
        article_data: JSON string with compound, abstract, and pmid fields

    Returns:
        JSON string with extracted interactions and confidence scores
    """
    try:
        data = json.loads(article_data)
        compound = data.get("compound", "")
        abstract = data.get("abstract", "")
        pmid = data.get("pmid", "")

        if not abstract or len(abstract) < 50:
            return json.dumps({"pmid": pmid, "interactions": []})

        llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.1)
        prompt = f"""You are a toxicology and molecular biology expert. Extract biological and toxicological information about {compound} from the following abstract.

ABSTRACT:
{abstract}

Extract the following information if present:
1. Proteins/enzymes that interact with {compound}
2. Genes that are affected or regulated
3. Cellular pathways impacted
4. Organs or systems affected
5. Toxicological mechanisms or effects
6. Molecular targets
7. the oragnism or specie

CRITICAL: You MUST respond with ONLY valid JSON. No explanations, no markdown, just raw JSON.

If you find relevant information, use this format:
{{"interactions": [{{"type": "protein/gene/pathway/mechanism/organ", "name": "specific_name", "interaction": "how_it_interacts_or_is_affected","Organism or specie":"was_it human_zebrahfish_rats_or_mice", "confidence": 0.9}}, ...]}}

If NO relevant information found:
{{"interactions": []}}

Remember: Only respond with JSON, nothing else."""

        response = llm.invoke([HumanMessage(content=prompt)])

        # Clean up response - remove markdown code blocks if present
        response_text = response.content.strip()
        if response_text.startswith("```"):
            response_text = response_text.split("```")[1]
            if response_text.startswith("json"):
                response_text = response_text[4:]
            response_text = response_text.strip()

        try:
            result = json.loads(response_text)
        except:
            result = {"interactions": []}

        result["pmid"] = pmid
        return json.dumps(result)
    except Exception as e:
        return json.dumps({"pmid": pmid, "interactions": []})


In [ ]:
# ============================================================================
# Cell 5: Nodes
# ============================================================================
def search_node(state):
    compound = state.get("compound_name", "")

    # Build a more targeted search query
    # Prioritizes toxicology, molecular interaction, and mechanism studies
    search_query = f'"{compound}" AND (toxicity OR toxicology OR "molecular target" OR protein OR gene OR pathway OR pharmacology OR mechanism OR hepatotoxicity OR nephrotoxicity OR neurotoxicity OR genotoxicity OR "drug metabolism" OR metabolism OR binding OR inhibitor OR receptor)'

    print(f"\n🔍 {compound}")
    print(f"   Query: {search_query[:80]}...")
    result = json.loads(search_pubmed.invoke({"compound": search_query, "max_results": 50}))
    return {**state, "pmids": result.get("pmids", []), "search_query": search_query}

def fetch_node(state):
    pmids = state.get("pmids", [])
    if not pmids:
        print("\n📥 No PMIDs to fetch")
        return state
    print(f"\n📥 Fetching...")
    result = json.loads(fetch_pubmed_articles.invoke(json.dumps(pmids)))
    return {**state, "articles": result.get("articles", [])}

def extract_node(state):
    articles, compound = state.get("articles", []), state.get("compound_name", "")
    if not articles:
        print("\n🧠 No articles to extract")
        return state
    print(f"\n🧠 Extracting interactions...")
    all_interactions = []

    # Process all articles (or limit to 25 for speed)
    max_articles = min(len(articles), 30)
    for i, article in enumerate(articles[:max_articles]):
        if not article.get("abstract"):
            continue
        print(f"  [{i+1}/{max_articles}]...", end=" ", flush=True)
        result = json.loads(extract_with_llm.invoke(json.dumps({
            "compound": compound,
            "abstract": article["abstract"],
            "pmid": article["pmid"]
        })))
        interactions = result.get("interactions", [])
        if interactions:
            print(f"✓ ({len(interactions)} found)")
            for inter in interactions:
                all_interactions.append({"pmid": article["pmid"], "title": article.get("title", ""), **inter})
        else:
            print("✗")
        time.sleep(0.5)  # Reduced from 1s

    return {**state, "interactions": all_interactions}


In [ ]:
# ============================================================================
# Cell 6: Create Agent
# ============================================================================
def create_agent():
    graph = StateGraph(CompoundState)
    graph.add_node("search", search_node)
    graph.add_node("fetch", fetch_node)
    graph.add_node("extract", extract_node)
    graph.add_edge(START, "search")
    graph.add_edge("search", "fetch")
    graph.add_edge("fetch", "extract")
    graph.add_edge("extract", END)
    return graph.compile()

agent = create_agent()
print("✅ Agent ready!\n")


In [ ]:

# ============================================================================
# Cell 7: Search function
# ============================================================================
def search_compound(compound_name: str):
    print(f"\n{'='*70}\n  ANALYZING: {compound_name}\n{'='*70}")
    initial_state = {
        "messages": [HumanMessage(content=f"Analyze {compound_name}")],
        "compound_name": compound_name,
        "search_query": "",
        "pmids": [],
        "articles": [],
        "interactions": []
    }
    result = agent.invoke(initial_state)
    interactions = result.get("interactions", [])

    print(f"\n{'='*70}")
    print(f"  RESULTS: {len(interactions)} interactions extracted")
    print(f"{'='*70}\n")

    if interactions:
        df = pd.DataFrame(interactions)
        # Display relevant columns
        display_cols = [col for col in ["compound_name", "pmid", "title", "name", "type", "interaction", "confidence"] if col in df.columns]
        print(df[display_cols].head(25).to_string(index=False))
        print(f"\n✅ Saved to {compound_name.lower().replace(' ', '_')}_interactions.csv")
        df.to_csv(f"{compound_name.lower().replace(' ', '_')}_interactions.csv", index=False)
        return df
    else:
        print("⚠️ No interactions found. Try adjusting the compound name or search terms.")
    return pd.DataFrame()



✅ Agent ready!



In [ ]:
def batch_search_compounds(csv_file_path: str, compound_column: str = "compound",
                          output_file: str = "all_interactions.csv",
                          start_index: int = 0,
                          limit: int = None):
    """Process multiple compounds from CSV and save all interactions to one file."""

    # Load CSV with different encodings
    print(f"\n📂 Loading: {csv_file_path}")

    encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252', 'utf-16']
    df_input = None

    for encoding in encodings:
        try:
            df_input = pd.read_csv(csv_file_path, encoding=encoding)
            print(f"✓ Loaded {len(df_input)} rows (encoding: {encoding})")
            break
        except Exception as e:
            continue

    if df_input is None:
        print("❌ Could not read CSV file with any encoding!")
        return None

    # Check column exists
    if compound_column not in df_input.columns:
        print(f"❌ Column '{compound_column}' not found!")
        print(f"Available: {list(df_input.columns)}")
        return None

    # Get unique compounds
    compounds = df_input[compound_column].dropna().unique().tolist()
    print(f"\n{'='*70}")
    print(f"  BATCH PROCESSING: {len(compounds)} unique compounds")
    print(f"{'='*70}\n")

    # Apply limits
    if limit:
        compounds = compounds[start_index:start_index + limit]
    else:
        compounds = compounds[start_index:]

    all_interactions = []
    successful = 0
    failed = 0

    # Process each compound
    for idx, compound in enumerate(compounds, start=start_index+1):
        try:
            print(f"[{idx}/{len(compounds) + start_index}] {compound}...", end=" ", flush=True)

            initial_state = {
                "messages": [HumanMessage(content=f"Analyze {compound}")],
                "compound_name": str(compound),
                "search_query": "",
                "pmids": [],
                "articles": [],
                "interactions": []
            }
            result = agent.invoke(initial_state)
            interactions = result.get("interactions", [])

            if interactions:
                print(f"✓ ({len(interactions)})")
                all_interactions.extend(interactions)
                successful += 1
            else:
                print("✗")
                failed += 1

            # Checkpoint every 10
            if idx % 10 == 0:
                df_temp = pd.DataFrame(all_interactions)
                df_temp.to_csv(output_file, index=False)
                print(f"   💾 Saved {len(all_interactions)} interactions\n")

        except Exception as e:
            print(f"❌ {str(e)[:40]}")
            failed += 1

    # Final save
    print(f"\n{'='*70}")
    print(f"  COMPLETE: {successful} success, {failed} failed")
    print(f"  Total: {len(all_interactions)} interactions")
    print(f"{'='*70}\n")

    if all_interactions:
        df_results = pd.DataFrame(all_interactions)
        df_results.to_csv(output_file, index=False)
        print(f"✅ Saved to: {output_file}\n")
        return df_results

    return pd.DataFrame()

In [ ]:
# Test with just FIRST 50 compounds (quick test)
results_df = batch_search_compounds(
    csv_file_path="/content/ALL DATA.csv",
    compound_column="compound_name",
    output_file="all_interactions_sample.csv",
    limit=2
)


📂 Loading: /content/ALL DATA.csv
✓ Loaded 1959 rows (encoding: latin-1)

  BATCH PROCESSING: 1836 unique compounds

[1/2] Acifluorfen... 
🔍 Acifluorfen
   Query: "Acifluorfen" AND (toxicity OR toxicology OR "molecular target" OR protein OR ge...
  ✓ Found 50 articles

📥 Fetching...
  ✓ Fetched 50 articles

🧠 Extracting interactions...
  [1/30]... ✗
  [2/30]... ✓ (4 found)
  [3/30]... ✓ (2 found)
  [4/30]... ✗
  [5/30]... ✓ (3 found)
  [6/30]... ✓ (2 found)
  [7/30]... ✓ (1 found)
  [8/30]... ✓ (5 found)
  [9/30]... ✓ (4 found)
  [10/30]... ✓ (4 found)
  [11/30]... ✓ (4 found)
  [12/30]... ✓ (11 found)
  [13/30]... ✗
  [14/30]... ✓ (2 found)
  [15/30]... ✓ (2 found)
  [16/30]... ✓ (18 found)
  [17/30]... ✓ (5 found)
  [18/30]... ✓ (12 found)
  [19/30]... ✗
  [20/30]... ✗
  [21/30]... ✓ (2 found)
  [22/30]... ✗
  [23/30]... ✗
  [24/30]... ✓ (5 found)
  [25/30]... ✓ (12 found)
  [26/30]... ✗
  [27/30]... ✓ (2 found)
  [28/30]... ✗
  [29/30]... ✓ (7 found)
  [30/30]... ✓ (3 found)
✓ (110)

In [ ]:
# ============================================================================
# Cell 8: RUN THIS IN NEW CELL:
# ============================================================================
results = search_compound("acetaminophen")
results = search_compound("caffeine")
results = search_compound("nicotine")


  ANALYZING: acetaminophen

🔍 acetaminophen
   Query: "acetaminophen" AND (toxicity OR toxicology OR "molecular target" OR protein OR ...
  ✓ Found 30 articles

📥 Fetching...
  ✓ Fetched 27 articles

🧠 Extracting interactions...
  [1/25]... ✓ (7 found)
  [2/25]... ✗
  [3/25]... ✗
  [4/25]... ✓ (3 found)
  [5/25]... ✗
  [6/25]... ✗
  [7/25]... ✓ (13 found)
  [8/25]... ✓ (17 found)
  [9/25]... ✗
  [10/25]... ✗
  [11/25]... ✗
  [12/25]... ✓ (3 found)
  [13/25]... ✓ (2 found)
  [14/25]... ✗
  [15/25]... ✗
  [16/25]... ✓ (2 found)
  [17/25]... ✗
  [18/25]... ✗
  [19/25]... ✗
  [20/25]... ✓ (13 found)
  [21/25]... ✗
  [22/25]... ✗
  [23/25]... ✓ (8 found)
  [24/25]... ✗
  [25/25]... ✓ (10 found)

  RESULTS: 78 interactions extracted

    pmid                                                                                                                                        title                                                                  name                    type                  

NON BATCH SERACH

OLD CODE

In [ ]:
"""
IMPROVED AGENT - Better search queries and extraction prompts
"""

# ============================================================================
# Cell 1: Install
# ============================================================================
import subprocess
subprocess.check_call(['pip', 'install', '-q',
    'langchain', 'langchain-google-genai', 'langgraph',
    'pandas', 'google-generativeai', 'requests'])

# ============================================================================
# Cell 2: Setup - CHANGE YOUR EMAIL HERE!!!
# ============================================================================
import os, json, re, time, requests, pandas as pd
from typing import TypedDict, Annotated, Sequence, List, Dict, Any
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
os.environ["NCBI_EMAIL"] = "Ukaegbudaniel33@gmail.com"  # ← CHANGE THIS!

print(f"✅ Email: {os.getenv('NCBI_EMAIL')}")

# ============================================================================
# Cell 3: State
# ============================================================================
class CompoundState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    compound_name: str
    search_query: str
    pmids: List[str]
    articles: List[Dict[str, Any]]
    interactions: List[Dict[str, Any]]

# ============================================================================
# Cell 4: Tools
# ============================================================================
import xml.etree.ElementTree as ET

@tool
def search_pubmed(compound: str, max_results: int = 50) -> str:
    """Search PubMed for articles on a chemical compound with toxicology focus.

    Args:
        compound: The search query for PubMed (can include boolean operators)
        max_results: Maximum number of results to return (default: 30)

    Returns:
        JSON string with list of PubMed IDs (pmids)
    """
    try:
        email = os.getenv("NCBI_EMAIL")
        if not email or "example" in email:
            raise ValueError("❌ REPLACE YOUR EMAIL IN THE CODE!")
        url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        params = {"db": "pubmed", "term": compound, "retmax": min(max_results, 100),
                  "rettype": "xml", "tool": "agent", "email": email}
        response = requests.get(url, params=params, timeout=20)

        root = ET.fromstring(response.text)
        pmids = [id_elem.text for id_elem in root.findall(".//Id")]

        print(f"  ✓ Found {len(pmids)} articles")
        return json.dumps({"pmids": pmids})
    except Exception as e:
        print(f"  ❌ {e}")
        return json.dumps({"pmids": []})

@tool
def fetch_pubmed_articles(pmids_list: str) -> str:
    """Fetch full article metadata from PubMed using PMIDs.

    Args:
        pmids_list: JSON string containing list of PubMed IDs

    Returns:
        JSON string with article titles and abstracts
    """
    try:
        pmids = json.loads(pmids_list)
        if not pmids:
            return json.dumps({"articles": []})
        email = os.getenv("NCBI_EMAIL")
        articles = []
        for i in range(0, len(pmids), 50):
            batch = pmids[i:i+50]
            url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
            params = {"db": "pubmed", "id": ",".join(batch), "rettype": "xml", "email": email}
            response = requests.get(url, params=params, timeout=20)

            root = ET.fromstring(response.text)
            for article in root.findall(".//PubmedArticle"):
                pmid_elem = article.find(".//PMID")
                title_elem = article.find(".//ArticleTitle")
                abstract_elem = article.find(".//AbstractText")

                pmid = pmid_elem.text if pmid_elem is not None else ""
                title = title_elem.text if title_elem is not None else ""
                abstract = abstract_elem.text if abstract_elem is not None else ""

                if pmid and abstract:
                    articles.append({"pmid": pmid, "title": title, "abstract": abstract})

            time.sleep(0.5)
        print(f"  ✓ Fetched {len(articles)} articles")
        return json.dumps({"articles": articles})
    except Exception as e:
        print(f"  ❌ {e}")
        return json.dumps({"articles": []})

@tool
def extract_with_llm(article_data: str) -> str:
    """Extract molecular interactions and toxicological mechanisms from an article abstract using LLM.

    Args:
        article_data: JSON string with compound, abstract, and pmid fields

    Returns:
        JSON string with extracted interactions and confidence scores
    """
    try:
        data = json.loads(article_data)
        compound = data.get("compound", "")
        abstract = data.get("abstract", "")
        pmid = data.get("pmid", "")

        if not abstract or len(abstract) < 50:
            return json.dumps({"pmid": pmid, "interactions": []})

        llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.1)
        prompt = f"""You are a toxicology and molecular biology expert. Extract biological and toxicological information about {compound} from the following abstract.

ABSTRACT:
{abstract}

Extract the following information if present:
1. Proteins/enzymes that interact with {compound}
2. Genes that are affected or regulated
3. Cellular pathways impacted
4. Organs or systems affected
5. Toxicological mechanisms or effects
6. Molecular targets

CRITICAL: You MUST respond with ONLY valid JSON. No explanations, no markdown, just raw JSON.

If you find relevant information, use this format:
{{"interactions": [{{"type": "protein/gene/pathway/mechanism/organ", "name": "specific_name", "interaction": "how_it_interacts_or_is_affected", "confidence": 0.9}}, ...]}}

If NO relevant information found:
{{"interactions": []}}

Remember: Only respond with JSON, nothing else."""

        response = llm.invoke([HumanMessage(content=prompt)])

        # Clean up response - remove markdown code blocks if present
        response_text = response.content.strip()
        if response_text.startswith("```"):
            response_text = response_text.split("```")[1]
            if response_text.startswith("json"):
                response_text = response_text[4:]
            response_text = response_text.strip()

        try:
            result = json.loads(response_text)
        except:
            result = {"interactions": []}

        result["pmid"] = pmid
        return json.dumps(result)
    except Exception as e:
        return json.dumps({"pmid": pmid, "interactions": []})

# ============================================================================
# Cell 5: Nodes
# ============================================================================
def search_node(state):
    compound = state.get("compound_name", "")

    # Build a more targeted search query
    # Prioritizes toxicology, molecular interaction, and mechanism studies
    search_query = f'"{compound}" AND (toxicity OR toxicology OR "molecular target" OR protein OR gene OR pathway OR pharmacology OR mechanism OR hepatotoxicity OR nephrotoxicity OR neurotoxicity OR genotoxicity OR "drug metabolism" OR metabolism OR binding OR inhibitor OR receptor)'

    print(f"\n🔍 {compound}")
    print(f"   Query: {search_query[:80]}...")
    result = json.loads(search_pubmed.invoke({"compound": search_query, "max_results": 30}))
    return {**state, "pmids": result.get("pmids", []), "search_query": search_query}

def fetch_node(state):
    pmids = state.get("pmids", [])
    if not pmids:
        print("\n📥 No PMIDs to fetch")
        return state
    print(f"\n📥 Fetching...")
    result = json.loads(fetch_pubmed_articles.invoke(json.dumps(pmids)))
    return {**state, "articles": result.get("articles", [])}

def extract_node(state):
    articles, compound = state.get("articles", []), state.get("compound_name", "")
    if not articles:
        print("\n🧠 No articles to extract")
        return state
    print(f"\n🧠 Extracting interactions...")
    all_interactions = []

    # Process all articles (or limit to 25 for speed)
    max_articles = min(len(articles), 25)
    for i, article in enumerate(articles[:max_articles]):
        if not article.get("abstract"):
            continue
        print(f"  [{i+1}/{max_articles}]...", end=" ", flush=True)
        result = json.loads(extract_with_llm.invoke(json.dumps({
            "compound": compound,
            "abstract": article["abstract"],
            "pmid": article["pmid"]
        })))
        interactions = result.get("interactions", [])
        if interactions:
            print(f"✓ ({len(interactions)} found)")
            for inter in interactions:
                all_interactions.append({"pmid": article["pmid"], "title": article.get("title", ""), **inter})
        else:
            print("✗")
        time.sleep(0.5)  # Reduced from 1s

    return {**state, "interactions": all_interactions}

# ============================================================================
# Cell 6: Create Agent
# ============================================================================
def create_agent():
    graph = StateGraph(CompoundState)
    graph.add_node("search", search_node)
    graph.add_node("fetch", fetch_node)
    graph.add_node("extract", extract_node)
    graph.add_edge(START, "search")
    graph.add_edge("search", "fetch")
    graph.add_edge("fetch", "extract")
    graph.add_edge("extract", END)
    return graph.compile()

agent = create_agent()
print("✅ Agent ready!\n")

# ============================================================================
# Cell 7: Search function
# ============================================================================
def search_compound(compound_name: str):
    print(f"\n{'='*70}\n  ANALYZING: {compound_name}\n{'='*70}")
    initial_state = {
        "messages": [HumanMessage(content=f"Analyze {compound_name}")],
        "compound_name": compound_name,
        "search_query": "",
        "pmids": [],
        "articles": [],
        "interactions": []
    }
    result = agent.invoke(initial_state)
    interactions = result.get("interactions", [])

    print(f"\n{'='*70}")
    print(f"  RESULTS: {len(interactions)} interactions extracted")
    print(f"{'='*70}\n")

    if interactions:
        df = pd.DataFrame(interactions)
        # Display relevant columns
        display_cols = [col for col in ["compound_name", "pmid", "title", "name", "type", "interaction", "confidence"] if col in df.columns]
        print(df[display_cols].head(25).to_string(index=False))
        print(f"\n✅ Saved to {compound_name.lower().replace(' ', '_')}_interactions.csv")
        df.to_csv(f"{compound_name.lower().replace(' ', '_')}_interactions.csv", index=False)
        return df
    else:
        print("⚠️ No interactions found. Try adjusting the compound name or search terms.")
    return pd.DataFrame()



ModuleNotFoundError: No module named 'langchain_core.pydantic_v1'

# Task
Update the installation cell `PQlKnTa9QsXU` to include `langchain-community` in both the `pip uninstall` and `pip install --upgrade` commands, and then execute this modified cell to resolve the `ModuleNotFoundError`.

## Update Installation Cell

### Subtask:
Modify cell PQlKnTa9QsXU to include langchain-community in both the pip uninstall and pip install --upgrade commands to ensure all langchain ecosystem packages are updated to mutually compatible versions.


**Reasoning**:
The user wants to modify the specified cell to include 'langchain-community' in both the uninstall and install commands. I will provide the updated code for cell PQlKnTa9QsXU.



In [ ]:
import subprocess
# Uninstall problematic packages to ensure a clean slate before installing specific versions
subprocess.check_call(['pip', 'uninstall', '-y', 'langchain', 'langchain-google-genai', 'langchain-core', 'langgraph', 'langchain-community'])
# Install the latest compatible versions to avoid issues with old, pinned versions
subprocess.check_call(['pip', 'install', '-q', '--upgrade',
    'langchain', 'langchain-google-genai', 'langchain-core', 'langgraph', 'langchain-community',
    'pandas', 'google-generativeai', 'requests'])

0

## Execute Installation Cell

### Subtask:
Execute the modified installation cell to apply the changes and attempt to resolve the ModuleNotFoundError.


## Summary:

### Data Analysis Key Findings
*   The installation cell `PQlKnTa9QsXU` was successfully updated to include `langchain-community` in both the `pip uninstall` and `pip install --upgrade` commands.
*   The update operation completed without errors, as confirmed by the output `{"stream": {"stdout": "0"}}`.

### Insights or Next Steps
*   The modified installation cell, which now includes `langchain-community` in its `pip uninstall` and `pip install --upgrade` commands, should be executed to apply the package updates. This action is crucial for resolving the `ModuleNotFoundError` by ensuring all `langchain` ecosystem packages are updated to mutually compatible versions.
